In [105]:
import pandas as pd
import numpy as np
import nltk
import re 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


In [106]:
data = pd.read_csv("Sentiment Analysis/all-data.csv",encoding="latin")
data.head()

,Sentiment,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [107]:
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from  nltk.stem import SnowballStemmer
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
#Object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def text_process(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
data["Text1"] = data["Text"].apply(text_process)


In [108]:
data.head()

,Sentiment,Text,Text1
0,neutral,"According to Gran , the company has no plans t...",according gran company plans move production r...
1,neutral,Technopolis plans to develop in stages an area...,technopolis plans develop stages area less 100...
2,negative,The international electronic industry company ...,international electronic industry company elco...
3,positive,With the new production plant the company woul...,new production plant company would increase ca...
4,positive,According to the company 's updated strategy f...,according company updated strategy years 2009 ...


In [109]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data["Text1"], data['Sentiment'], test_size = 0.20, random_state = 121)

def train_test(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    return train_acc, test_acc


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfVect = TfidfVectorizer()
X_train = tfVect.fit_transform(X_train)
X_test = tfVect.transform(X_test)

In [112]:
# with open("sentimentTFdict.h5","wb") as vect:
#     pickle.dump(tfVect,vect)

In [76]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.calibration import CalibratedClassifierCV

#svc = SVC(probability=True)
lsvc = LinearSVC(random_state=123)
lsvc = CalibratedClassifierCV(lsvc)
#rforest = RandomForestClassifier(random_state=123)
#mnb = MultinomialNB()
#dtree = DecisionTreeClassifier()

clifs =[lsvc]# [svc,rforest, lsvc,mnb]# dtree]

# train and test them 
print("| {:25} | {} | {} |".format("Classifier", "Training Accuracy", "Test Accuracy"))
print("| {} | {} | {} |".format("-"*25, "-"*17, "-"*13))
for clf in clifs: 
    clf_name = clf.__class__.__name__
    train_acc, test_acc = train_test(clf, X_train, X_test, y_train, y_test)
    print("| {:25} | {:17.7f} | {:13.7f} |".format(clf_name, train_acc, test_acc))

| Classifier                | Training Accuracy | Test Accuracy |
| ------------------------- | ----------------- | ------------- |
| CalibratedClassifierCV    |         0.9819035 |     0.8766636 |


In [113]:
import pickle
with open("SentimentLSVC.h5" ,"wb") as model:
    pickle.dump(clf,model)

In [77]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,clf.predict(X_test)))

              precision    recall  f1-score   support

    negative       0.89      0.88      0.89      5108
     neutral       0.79      0.80      0.79       592
    positive       0.87      0.88      0.88      5270

    accuracy                           0.88     10970
   macro avg       0.85      0.85      0.85     10970
weighted avg       0.88      0.88      0.88     10970



In [95]:
def textEmotions(clf,text):
    output = []
    emotions = ["Negative","Neutral","Positive"]
    emotions.sort()

    features = tfVect.transform([text])
    prediction_prob = clf.predict_proba(features)[0]
    prediction = clf.predict(features)[0]
    print(text)
    print("-"*148)
    print(prediction)
    for i in range(len(prediction_prob)):
        print(emotions[i],np.round(prediction_prob[i],3))        
        output.append([emotions[i],np.round(prediction_prob[i],3)])
    return(output)

In [114]:
import random
index = random.randint(1,50000)
text = data["Text1"][index]
print(data["Sentiment"][index])
Emotionpred = textEmotions(clf,text)

negative
young couple mandy pullman mitch martin roy seeley matt birman relaxing beach small town galen decide start playing practical jokes mandy hides old run cabin attacked raped unknown assailant roy tries help hearing screams killed dr sam cordell john cassavetes daughter jenny erin flannery new galen death sam wife sam called action police chief hank walden john ireland mandy roy found performs autopsy roy treats mandy horrific injuries soon curator local museum named carolyn davis denise furgusson also attacked raped local journalist named laura kincaid kerrie keane reports events suggests sam similar string rapes murders occurred town 30 years earlier rapes murders occur meanwhile jenny boyfriend tim galen duncan macintosh strange dreams nightmares convinced something horrific acts tim story digging towns past makes sam become convinced existence creature known incubus shape shifting demonic entity exists reproduce directed john hough one seriously dull horror film script georg